In [3]:
# validate_data.py  ←  UPDATED FULL VERSION WITH EXCEL SUMMARY

import sqlite3
import pandas as pd
import logging

DB = r"E:\work\DEPI\graduation promax\data\warehouse_final\ivf_star_schema.db"
LOG = r"E:\work\DEPI\graduation promax\src\ETL\logs\validation_log.txt"

logging.basicConfig(
    filename=LOG,
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)

STATUS_FLAG = 1     # 1 = OK, -1 = Validation issues


# ===========================================================
# REQUIRED COLUMNS (ONLY HARD-REQUIRED)
# ===========================================================
REQUIRED_FACT_COLS = {
    "fact_ivf_cycle": ["case_id", "female_id", "male_id"],
    "fact_transfer": ["case_id"],
    "fact_transfer_embryo": ["transfer_sk"]   # embryo_id ALLOWED TO BE NULL
}


def run_validation():
    global STATUS_FLAG
    conn = sqlite3.connect(DB)

    print("\n🔎 RUNNING FULL VALIDATION...\n")

    # ------------------------------------------------------
    # 1) List all tables
    # ------------------------------------------------------
    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table'",
        conn
    )
    tbl_list = list(tables["name"])

    print("🧾 Tables in DB:\n", tables, "\n")
    logging.info(f"Tables found:\n{tables}")

    # ------------------------------------------------------
    # 2) Check for duplicates
    # ------------------------------------------------------
    duplicate_results = []

    for tbl in tbl_list:
        df = pd.read_sql(f"SELECT * FROM {tbl}", conn)
        dup_count = df.duplicated().sum()

        duplicate_results.append({"table": tbl, "duplicate_rows": dup_count})

        if dup_count > 0:
            STATUS_FLAG = -1

        print(f"🔍 Duplicates in {tbl} → {dup_count}")
        logging.info(f"Duplicates in {tbl}: {dup_count}")

    # ------------------------------------------------------
    # 3) NULL checks
    # ------------------------------------------------------
    print("\n🕳 NULL VALUES CHECK:")
    null_rows_all = []

    for tbl in tbl_list:
        df = pd.read_sql(f"SELECT * FROM {tbl}", conn)
        nulls = df.isnull().sum()

        # Validation: Only check required columns
        if tbl in REQUIRED_FACT_COLS:
            for col in REQUIRED_FACT_COLS[tbl]:
                if col in df.columns and df[col].isnull().sum() > 0:
                    STATUS_FLAG = -1
                    print(f"❌ ERROR: {tbl}.{col} has NULL values — REQUIRED!")

        # Store for Excel
        for col, n in nulls.items():
            null_rows_all.append({
                "table": tbl,
                "column": col,
                "null_count": n
            })

        print(f"\n⚠ Table: {tbl}\n{nulls}")
        logging.info(f"NULLS in {tbl}:\n{nulls}")

    # ------------------------------------------------------
    # 4) Foreign Key Checks
    # ------------------------------------------------------
    print("\n🔗 FOREIGN KEY CHECKS:")

    fk_checks = {
        "fact_ivf_cycle → dim_time": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_ivf_cycle fc
            LEFT JOIN dim_time dt 
                ON fc.cycle_start_time_id = dt.time_id
            WHERE fc.cycle_start_time_id IS NOT NULL
              AND dt.time_id IS NULL
        """,

        "fact_transfer → dim_time": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_transfer ft
            LEFT JOIN dim_time dt 
                ON ft.transfer_time_fk = dt.time_id
            WHERE ft.transfer_time_fk IS NOT NULL
              AND dt.time_id IS NULL
        """,

        "fact_transfer → dim_doctor": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_transfer f
            LEFT JOIN dim_doctor d 
                ON f.doctor_id = d.doctor_id
            WHERE f.doctor_id IS NOT NULL
              AND d.doctor_id IS NULL
        """,

        "fact_transfer_embryo → fact_transfer": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_transfer_embryo fe
            LEFT JOIN fact_transfer ft 
                ON fe.transfer_sk = ft.transfer_sk
            WHERE fe.transfer_sk IS NOT NULL
              AND ft.transfer_sk IS NULL
        """,

        "fact_transfer_embryo → dim_embryo": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_transfer_embryo fe
            LEFT JOIN dim_embryo e 
                ON fe.embryo_id = e.embryo_id
            WHERE fe.embryo_id IS NOT NULL
              AND e.embryo_id IS NULL
        """
    }

    fk_summary = []

    for title, q in fk_checks.items():
        df_fk = pd.read_sql(q, conn)
        invalid_fk = int(df_fk.iloc[0, 0])

        fk_summary.append({"check": title, "invalid_count": invalid_fk})

        if invalid_fk > 0:
            STATUS_FLAG = -1

        print(f"{title} → {invalid_fk}")
        logging.info(f"{title}: {invalid_fk}")

    # ------------------------------------------------------
    # 5) ID format checks SKIPPED
    # ------------------------------------------------------
    print("\n🧬 ID FORMAT CHECK SKIPPED (IDs are nullable/custom).")
    logging.info("ID regex checks skipped.")

    # ------------------------------------------------------
    # 6) EXPORT EXCEL SUMMARY
    # ------------------------------------------------------
    print("\n📊 Generating Excel summary...")

    summary_path = r"E:\work\DEPI\graduation promax\src\ETL\logs\validation_summary.xlsx"

    with pd.ExcelWriter(summary_path, engine="openpyxl") as writer:
        tables.to_excel(writer, sheet_name="tables", index=False)

        pd.concat(
            [pd.DataFrame(null_rows_all)],
            ignore_index=True
        ).to_excel(writer, sheet_name="nulls", index=False)

        pd.DataFrame(fk_summary).to_excel(writer, sheet_name="fk_checks", index=False)

        pd.DataFrame(duplicate_results).to_excel(
            writer, sheet_name="duplicates", index=False
        )

    print(f"\n📄 Excel summary saved at:\n{summary_path}")
    logging.info(f"Excel summary saved at: {summary_path}")

    # ------------------------------------------------------
    # FINAL STATUS
    # ------------------------------------------------------
    print(f"\n📌 FINAL STATUS = {STATUS_FLAG}")
    logging.info(f"FINAL STATUS = {STATUS_FLAG}")

    conn.close()
    return STATUS_FLAG


if __name__ == "__main__":
    run_validation()
    print("\n📌 Validation Finished — Check logs!")



🔎 RUNNING FULL VALIDATION...

🧾 Tables in DB:
                     name
0             dim_doctor
1        sqlite_sequence
2               dim_time
3         fact_ivf_cycle
4          fact_transfer
5   fact_transfer_embryo
6             dim_female
7               dim_male
8           dim_protocol
9            dim_outcome
10            dim_embryo 

🔍 Duplicates in dim_doctor → 0
🔍 Duplicates in sqlite_sequence → 0
🔍 Duplicates in dim_time → 0
🔍 Duplicates in fact_ivf_cycle → 0
🔍 Duplicates in fact_transfer → 0
🔍 Duplicates in fact_transfer_embryo → 0
🔍 Duplicates in dim_female → 0
🔍 Duplicates in dim_male → 0
🔍 Duplicates in dim_protocol → 0
🔍 Duplicates in dim_outcome → 0
🔍 Duplicates in dim_embryo → 0

🕳 NULL VALUES CHECK:

⚠ Table: dim_doctor
doctor_id      0
doctor_name    0
dtype: int64

⚠ Table: sqlite_sequence
name    0
seq     0
dtype: int64

⚠ Table: dim_time
time_id       0
full_date     0
day           0
month         0
month_name    0
quarter       0
year          0
week    